In [243]:
# import packages

from dash import Dash, dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd
import numpy as np

# import data and assign a rank to certain metrics

resorts = (
    pd.read_csv("data/resorts.csv", encoding = "ISO-8859-1")
    .assign(
        country_elevation_rank = lambda x: x.groupby("Country", as_index=False)["Highest point"].rank(ascending=False),
        country_price_rank = lambda x: x.groupby("Country", as_index=False)["Price"].rank(ascending=False),
        country_slope_rank = lambda x: x.groupby("Country", as_index=False)["Total slopes"].rank(ascending=False),
        country_lift_rank = lambda x: x.groupby("Country", as_index=False)["Total lifts"].rank(ascending=False),
        global_elevation_rank = lambda x: x["Highest point"].rank(ascending=False),
        global_price_rank = lambda x: x["Price"].rank(ascending=False),
        global_slope_rank = lambda x: x["Total slopes"].rank(ascending=False),
        global_lift_rank = lambda x: x["Total lifts"].rank(ascending=False),
    ))


#resorts['Resort'] = resorts['Resort'].replace('[^\w\s]', ' ', regex=True)

# application

app = Dash(__name__, external_stylesheets = [dbc.themes.BOOTSTRAP])

load_figure_template("BOOTSTRAP")

app_style = {"backgroundColor": "#F5F7FB",
             "color": "#33394C",
            "text-align": "left",
            "height": "100vh"}

app.layout = (
html.Div(
    style = app_style,
    children = [
        dbc.NavbarSimple(
                    children=[dbc.NavItem(dbc.NavLink("GitHub", href="https://github.com/kieswetter-hub/Ski-resort-dashboard"))],
                            brand="Exploring Global Ski Resorts",
                            color="#4E7CFF",
                            dark=True),
        dbc.Tabs(id = "tabs",
        children = [
            dbc.Tab(label = "Resort Map",
                    tab_style={"margin": 0,
                               "width": "50%",
                               "font-size": "18px"},
                    label_style={"text-align": "center",
                                 "color": "#33394C"},
                    active_label_style={"background-color": "#F5F7FB"},
                            children = [
                                dbc.Container([
                                    dbc.Row([
                                        html.P(),
                                        html.H2(id="title", style = {"text-align": "center"}),
                                        html.P()]),
                                    dbc.Row([dbc.Col([
                                                      dbc.Card(style = {"text-align":"left",
                                                                        "backgroundColor": "#fefefe",
                                                                        "padding" : "20px"},
                                                        children = [
                                                            html.P("Select a price", style = {"font-size": 18}),
                                                            dcc.Slider(id = "maxprice",
                                                                       min = 0,
                                                                       max = 150,
                                                                       step = 25,
                                                                       value = 100,
                                                                       marks={i:{"label": f'${i}', 
                                                                                 "style": {"fontSize": 14}} 
                                                                                 for i in range(0, 151, 25)}),
                                                            html.Br(),
                                                            html.P("Select features", style = {"font-size": 18}),
                                                            dcc.Checklist(id = "selectFeatures",
                                                                          options = [
                                                                              {'label': '   Summer skiing', 'value': 'Summer skiing'},
                                                                              {'label': '   Night skiing', 'value': 'Nightskiing'},
                                                                              {'label': '   Snow parks', 'value': 'Snowparks'}],
                                                                              value = [])])], width = 4),
                                            dbc.Col([
                                                      dbc.Card(style = {"text-align":"center",
                                                                        "backgroundColor": "#fefefe"},
                                                        children = [
                                                            dcc.Graph(id = "map")])], width = 8)
                                            ]) # closes row
                                ]) # closes container
                        ]), # closes children tab 1                       
            dbc.Tab(label = "Country Profile",
                    tab_style={"margin": 0,
                               "width": "50%",
                               "font-size": "18px"},
                    label_style={"text-align": "center",
                                 "color": "#33394C"},
                    active_label_style={"background-color": "#F5F7FB"},
                            children = [
                                dbc.Container([
                                    dbc.Row([
                                        html.P(),
                                        html.H2(id="title2", style = {"text-align": "center"}),
                                        html.P()
                                        ]),
                                    dbc.Row([dbc.Col([
                                                      dbc.Card(style = {"text-align":"left",
                                                                        "backgroundColor": "#fefefe",
                                                                        "padding" : "20px"},
                                                        children = [
                                                                html.P("Select a continent", style = {"font-size": 18}),
                                                                dcc.Dropdown(id = "selectContinent",
                                                                             options =[{'label': c, 'value': c} for c in np.sort(resorts["Continent"].unique())],
                                                                             value = "Europe"),
                                                                html.Br(),
                                                                html.P("Select a country", style = {"font-size": 18}),
                                                                dcc.Dropdown(id="selectCountry",
                                                                             value = "Norway"),
                                                                html.Br(),
                                                                html.P("Select a metric", style = {"font-size": 18}),
                                                                dcc.Dropdown(id = "selectMetric",
                                                                             options = ["Price","Highest point","Total slopes","Total lifts"],
                                                                             value = "Price"),
                                                                html.Br(),                                                                                                                                                       
                                                                    ])], 
                                                                    width = 3),
                                            dbc.Col([
                                                      dbc.Card(style = {"text-align":"center",
                                                                        "backgroundColor": "#fefefe"},
                                                        children = [
                                                            dcc.Graph(id = "barchart", hoverData = {"points": [{"customdata": [""]}]})])], 
                                                            width = 7),
                                             dbc.Col([dbc.Card(
                                                 children = [
                                                 dbc.CardHeader("Local Rank (Global)"),
                                                 dbc.CardBody([
                                                     html.H5(id = "resort-name", style = {"padding-bottom": "10px"}),
                                                     html.P(id = "price-rank"),
                                                     html.P(id = "elevation-rank"),
                                                     html.P(id = "slope-rank"),
                                                     html.P(id = "lift-rank"),
                                                            ]),
                                                            ]
                                                      )], 
                                                      width = 2)
                                            ])])
                                ])
                    ])
        ])
)

@app.callback(
    Output("title", "children"),
    Output("map", "figure"),
    Input("maxprice", "value"),
    Input("selectFeatures", "value"),
)

def update_map(pricelimit, features):
    
    df_filtered = resorts.query("Price <= @pricelimit")

    title = f"Slope density of ski resorts under ${pricelimit}"

    if len(features) == 1:
        if "Nightskiing" in features:
            df_filtered = df_filtered.query('Nightskiing == "Yes"')
        elif "Summer skiing" in features:
            df_filtered = df_filtered.query('`Summer skiing` == "Yes"')
        else:
            df_filtered = df_filtered.query('Snowparks == "Yes"')
    elif len(features) == 2:
        if "Nightskiing" in features and "Summer skiing" in features :
            df_filtered = df_filtered.query("`Summer skiing` == 'Yes' and Nightskiing == 'Yes'")
        if "Nightskiing" in features and "Snowparks" in features :
            df_filtered = df_filtered.query("Snowparks == 'Yes' and Nightskiing == 'Yes'")
        if "Summer skiing" in features and "Snowparks" in features :
            df_filtered = df_filtered.query("`Summer skiing` == 'Yes' and Snowparks == 'Yes'")
    elif len(features) == 3:
            df_filtered = df_filtered.query("`Summer skiing` == 'Yes' and Nightskiing == 'Yes' and Snowparks == 'Yes'")
    else:
        df_filtered = df_filtered

    figure = (px.density_mapbox(
        df_filtered,
        lat = "Latitude",
        lon = "Longitude",
        z = "Total slopes",
        mapbox_style = "carto-positron",
        zoom = 2.5,
        center = {"lat": 52.5, "lon": 15},
        radius = 15,
        height = 500,
        hover_name="Resort",
        hover_data={'Latitude':False, 'Longitude':False})
        .update_layout(
            geo_bgcolor = "#F5F7FB",
            paper_bgcolor = "#fefefe"))
    
    return title, figure

@app.callback(
        Output("selectCountry", "options"),
        Input("selectContinent", "value"),
)

def populateCountryDropDown(continent):  
    if not continent:
        raise PreventUpdate  
    
    country_list = np.sort(resorts.query("Continent == @continent")["Country"].unique())

    return country_list


@app.callback(
    Output("title2", "children"),
    Output("barchart", "figure"),
    Input("selectCountry", "value"),
    Input("selectMetric", "value")
)

def update_bar(country, metric):
    if not country:
        raise PreventUpdate

    df_bar = resorts.query("Country == @country").sort_values(by = metric, ascending = False).head(10)
    df_bar
    
    title2 = f"Top resorts in {country} based on {metric.lower()}"

    figure2 = (px.bar(df_bar,
                      x = "Resort",
                      y = metric,
                      hover_name="Resort",
                      hover_data={'Resort':False},
                      custom_data=["Resort", "country_price_rank"])
                      .update_traces(
                          showlegend=False,
                          marker_color="#4E7CFF",
                          opacity = 0.85)
                      .update_xaxes(tickfont=dict(color="rgba(0,0,0,0)", size = 1)))
    
    return title2, figure2

@app.callback(
    Output("resort-name", "children"),
    Output("price-rank", "children"),
    Output("elevation-rank", "children"),
    Output("slope-rank", "children"),
    Output("lift-rank", "children"),
    Input("barchart", "hoverData")
)

def populate_report(hoverData):
    resortname = hoverData["points"][0]["customdata"][0]

    priceRankCountry = resorts.query("Resort == @resortname")["country_price_rank"].iloc[0].astype(int)
    elevationRankCountry = resorts.query("Resort == @resortname")["country_elevation_rank"].iloc[0].astype(int)
    slopeRankCountry = resorts.query("Resort == @resortname")["country_slope_rank"].iloc[0].astype(int)
    liftRankCountry = resorts.query("Resort == @resortname")["country_lift_rank"].iloc[0].astype(int)

    priceRankGlobal = resorts.query("Resort == @resortname")["global_price_rank"].iloc[0].astype(int)
    elevationRankGlobal = resorts.query("Resort == @resortname")["global_elevation_rank"].iloc[0].astype(int)
    slopeRankGlobal = resorts.query("Resort == @resortname")["global_slope_rank"].iloc[0].astype(int)
    liftRankGlobal = resorts.query("Resort == @resortname")["global_lift_rank"].iloc[0].astype(int)

    priceRank = f'Price: {priceRankCountry} ({priceRankGlobal})'
    elevationRank = f'Elevation: {elevationRankCountry} ({elevationRankGlobal})'
    slopeRank = f'Total slopes: {slopeRankCountry} ({slopeRankGlobal})'
    liftRank = f'Total lifts: {liftRankCountry} ({liftRankGlobal})'

    return resortname, priceRank, elevationRank, slopeRank, liftRank

if __name__ == "__main__":
    app.run_server(port = 8051, debug = True, jupyter_mode="external")

Dash app running on http://127.0.0.1:8051/


---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
Cell In[243], line 262, in populate_report(
    hoverData={'points': [{'customdata': ['']}]}
)
    250 @app.callback(
    251     Output("resort-name", "children"),
    252     Output("price-rank", "children"),
   (...)
    258 
    259 def populate_report(hoverData):
    260     resortname = hoverData["points"][0]["customdata"][0]
--> 262     priceRankCountry = resorts.query("Resort == @resortname")["country_price_rank"].iloc[0].astype(int)
        priceRankCountry = 8
        resorts =       ID                                         Resort   Latitude  \
0      1                                       Hemsedal  60.928244   
1      2                               Geilosiden Geilo  60.534526   
2      3                                           Golm  47.057810   
3      4                   Red Mountain Resort-Rossland  49.105520   
4    

In [159]:
resorts2 = (
    pd.read_csv("data/resorts.csv", encoding = "ISO-8859-1")
    .assign(
        country_elevation_rank = lambda x: x.groupby("Country", as_index=False)["Highest point"].rank(ascending=False),
        country_price_rank = lambda x: x.groupby("Country", as_index=False)["Price"].rank(ascending=False),
        country_slope_rank = lambda x: x.groupby("Country", as_index=False)["Total slopes"].rank(ascending=False),
        country_lift_rank = lambda x: x.groupby("Country", as_index=False)["Total lifts"].rank(ascending=False),
        global_elevation_rank = lambda x: x["Highest point"].rank(ascending=False),
        global_price_rank = lambda x: x["Price"].rank(ascending=False),
        global_slope_rank = lambda x: x["Total slopes"].rank(ascending=False),
        global_lift_rank = lambda x: x["Total lifts"].rank(ascending=False),
    ))

resorts2.sort_values(["country_elevation_rank"])

,ID,Resort,Latitude,Longitude,Country,Continent,Price,Season,Highest point,Lowest point,...,Nightskiing,Summer skiing,country_elevation_rank,country_price_rank,country_slope_rank,country_lift_rank,global_elevation_rank,global_price_rank,global_slope_rank,global_lift_rank
353,354,Szczyrk-Skrzyczne,49.707120,19.011709,Poland,Europe,20,December - April,1257,524,...,No,No,1.0,2.0,1.0,1.0,425.0,481.0,362.5,236.0
268,269,Bukovel,48.353783,24.412830,Ukraine,Europe,32,November - May,1372,890,...,Yes,No,1.0,1.0,1.0,1.0,405.0,414.5,209.5,207.0
49,50,Valle Nevado,-33.352964,-70.248678,Chile,South America,64,June - October,3670,2860,...,No,Yes,1.0,1.5,2.0,2.0,12.0,79.5,335.0,256.0
180,181,Sunshine Village,51.089525,-115.762202,Canada,North America,66,November - May,2730,1660,...,No,No,1.0,4.5,7.0,5.5,120.0,72.0,109.0,326.5
48,49,Les Deux alpes,45.009995,6.123761,France,Europe,49,"December - April, June - August, October - Nov...",3560,1280,...,Yes,Yes,1.0,18.0,18.5,20.0,17.0,198.5,45.0,47.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,205,Oberau (Wildscho?nau),47.423497,12.030743,Austria,Europe,30,December - March,1130,900,...,No,No,85.0,85.0,89.0,89.0,445.0,431.5,496.5,494.0
374,375,Gerlitzen,46.694760,13.914332,Austria,Europe,45,December - April,1000,520,...,No,No,86.0,46.0,49.0,40.5,461.5,254.5,326.5,207.0
415,416,Tauplitz-Bad Mitterndorf,47.577281,13.984784,Austria,Europe,41,December - April,896,812,...,No,No,87.0,64.0,43.5,38.0,475.0,322.0,310.0,195.5
457,458,Kellerjoch-Schwaz,47.330214,11.718045,Austria,Europe,29,April,857,545,...,No,No,88.0,88.0,83.0,81.5,481.0,442.5,467.5,456.0


In [225]:
metric = "Price"
country = "Norway"
continent = "Europe"
df_bar = resorts.query("Country == @country").sort_values(by = metric, ascending = False).head(10)
df_bar
    
title2 = f"Top 10 resorts in {country} based on {metric}"
title2

figure2 = (px.bar(df_bar,
              x = "Resort",
              y = metric).update_xaxes(tickfont=dict(color="rgba(0,0,0,0)")))

figure2



In [175]:
priceRankGlobal = resorts.query("Resort == 'Voss'")["country_price_rank"].iloc[0].astype(int)
priceRankCountry

8

In [139]:
resortname = "Voss"

elevation = resorts.query("Resort == @resortname")["country_elevation_rank"]
elevation

resorts.query("Resort == @resortname")



# country_elevation_rank 
# country_price_rank 
# country_slope_rank 
# ountry_lift_rank 

# global_elevation_rank 
# global_price_rank 
# global_slope_rank 
# global_lift_rank

,ID,Resort,Latitude,Longitude,Country,Continent,Price,Season,Highest point,Lowest point,...,Total lifts,Lift capacity,Child friendly,Snowparks,Nightskiing,Summer skiing,country_elevation_rank,country_price_rank,country_slope_rank,country_cannon_rank
5,6,Voss,60.683706,6.407905,Norway,Europe,43,November - May,960,284,...,11,11900,Yes,Yes,No,No,10.0,8.5,5.0,7.0
